In [1]:
def sequence_extractor(Species, Scaff, reverse_c, start, end, frame = 1, trans = 1 ):
    from Bio import SeqIO
    from Bio.Seq import Seq
    import os
    
    out_Seq = ''
    
    
    
    #Genome folder
    entries = os.listdir("/mnt/f/Genomes_2023-12-26/"+Species)
    
    #Get genome file from Genome folder
    for file_names  in entries:
        if ".nhr" in file_names:
            Genome_name = file_names[:-4]
            break
    
    #Read the genome file
    fasta_file = SeqIO.parse(("/mnt/f/Genomes_2023-12-26/"+Species+"/"+Genome_name),'fasta')
    
    #Extract the sequence 
    for record in fasta_file:
        if record.id == Scaff:
            sequence = str(record.seq)
            out_Seq = Seq(sequence[start-1:end])
            if reverse_c == 1:
                out_Seq = out_Seq.reverse_complement()
            break
                
    if len(out_Seq) < 10000: #fixing error due to mistake in typing the coordinate, change this for longer sequence
#        print (out_Seq)
        if frame == 1 :
            out_trans = out_Seq[0:]
        if frame == 2 :
            out_trans = out_Seq[1:]
        if frame == 3 :
            out_trans = out_Seq[2:]
    else:
        print ("too long")
        assert(False)
        
        
    #    break
    
    if trans == 1:
        return (out_trans.translate())
    else:
        return(out_Seq)
#

In [2]:
import os
from Bio import SeqIO

In [3]:
def get_sequence(Species, Scaff, reverse_c, start, end, frame = 1, trans = 1 ):
    
    #Genome folder
    entries = os.listdir("/mnt/f/Genomes_2023-12-26/"+Species)
    
    #Get genome file from Genome folder
    for file_names  in entries:
        if ".nhr" in file_names:
            Genome_name = file_names[:-4]
            break
    
    #Read the genome file
    fasta_file = SeqIO.parse(("/mnt/f/Genomes_2023-12-26/"+Species+"/"+Genome_name),'fasta')
    
    #Extract the sequence 
    for record in fasta_file:
        if record.id == Scaff:
            sequence = str(record.seq)
#             out_Seq = Seq(sequence[start-1:end])
#             if reverse_c == 1:
#                 sequence = out_Seq.reverse_complement()
            return(sequence)
    
#

In [4]:
#%reset -f

from Bio.Seq import Seq


family_group = "6.Heliconiinae_Danainae_Nymphalinae"
blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}"
query_location = f"{blast_location}/1.Query"
query_species = "Danaus_plexippus"
query_transcript_list = os.listdir(f"{query_location}/{query_species}")
if "desktop.ini" in query_transcript_list:
    query_transcript_list.remove("desktop.ini")

query_transcript = query_transcript_list[0]
#Get Species list, the list may be a list of files or a single species

species_list = os.listdir(f"{blast_location}/1.Blast_result")
# species_list = ["Pararge_aegeria"]
if "desktop.ini" in species_list:
    species_list.remove('desktop.ini')

#get list of query names

query_file = SeqIO.parse(f"{blast_location}/1.Query/{query_species}/{query_transcript}/query_protein.fa", 'fasta')
query_name_list = []

for records in query_file:
    query_name_list.append(records.id)



for species in species_list:
    # print(species_name)
    # species = species_name.split(".")[1]
    header = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length\n" 
    Output_Sequence = header
    scaff = "Intial_value"
    scaff_old = "Intial_value"
    old_end = 0
    
#Run for each query in the query list
    for i in range(len(query_name_list)):
        query_name = query_name_list[i]
#         print(f"Query name = {query_name}")
        Length_switch = "0"
        with open(f"{blast_location}/1.Blast_result/{species}/{species}_blast_out.txt",'r') as tblast_out:
            lines_in_file = tblast_out.readlines()
        #print(lines_in_file)

        result_section_switch = 0
        start_coor_switch = 0
        query_start_coor_switch = 0
        stop_coor_switch = 0
        error = "N"
        break_switch = 0

        start = 0
        stop = 0
        start_coor = 0
        stop_coor = 0
        query_length = 0

        for lines in lines_in_file:

#             print(lines)
            if query_name in lines:
#                 print(lines)
#                 assert False
            #Initialize that results can now be checked
                result_section_switch = 1
#                 query_species_split = lines.split(" ")[1].split("_")
#                 query_species = str(query_species_split[0].split(".")[1]+"_"+query_species_split[1].rstrip())
                print(query_species)
            if result_section_switch == 1 and "Lambda" in lines:
            #This block indicates end of the results block in blast output
                result_section_switch == 0
                
                break

            if result_section_switch == 1:
            #While checking the result
                if "Length=" in lines and Length_switch == "0":
                #Get query length from the blast output
                    
                    query_length = int(lines.split("=")[1].rstrip())
                    
                    Length_switch = 1 #Indicated length has been acquired
                    
                if ("Score" in lines or ">" in lines) and (start_coor_switch == 1):
    #                print (lines)
                    break
        
                if ">" in lines:
                #Start of the first result
                    scaff = lines.split(" ")[0][1:] #Scaffold from the result
                    if scaff_old != "Intial_value" and scaff_old != scaff:
                        error = "Y"
                    scaff_old = scaff
                    
                if "Query" in lines and "=" not in lines:
                #Read the query line in output
                    if query_start_coor_switch == 0:
#                        print(lines)
                        query_start_coor = int(lines.split(" ")[2])
                        query_start_coor_switch = 1
                        #Query start coordinate fixed
            
                    query_stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting query stop coordinates for multiline result
    #                print (stop_coor)
                    
                if "Sbjct" in lines:
                #Read the blast target line
                    if start_coor_switch == 0:
                        start_coor = int(lines.split(" ")[2])
                        start_coor_switch = 1
                    stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting target stop coordinates for multiline result
                
                

        
        if break_switch == 1:
            break

        if start_coor < stop_coor:
            complement = "0" #Forward complement
            
            length = (stop_coor-start_coor)/3
            start = start_coor
            stop = stop_coor

        elif start_coor > stop_coor:
            complement = "1" #Reverse complement
            length = (-stop_coor+start_coor)/3
            start = stop_coor
            stop = start_coor

        else:
            length = 0
            start = 000
            stop = 000
            query_start_coor = 000
            query_stop_coor = 000
            error = "Y"
            complement  = 0
        
        
        
        

        #Check if the length of target (blast hit) is significantly smaller than query
        if length < query_length - 0.2*query_length:
            error = "Y"

        old_trans = ''
#             print(start, stop)
        
    
    
        

#Check for Met or STOP at the start and end
        sequence = ''
        while True and (start != 0 or stop != 0):
            if (query_name == query_name_list[0] or query_name == query_name_list[-1]):
                frame = 1
#                 translated_sequence = (sequence_extractor(species, scaff, int(complement), start, stop,frame ))
                if sequence == '':
                    sequence = get_sequence(species, scaff, int(complement), start, stop,frame)
                if int(complement) == 1:
                    translated_sequence = Seq(sequence[start-1:stop]).reverse_complement().translate()
                else:
                    translated_sequence = Seq(sequence[start-1:stop]).translate()
                if translated_sequence == old_trans:
                    break
                    
#Check for Met at the beginning of first exon

                if query_name == query_name_list[0] :
                    if translated_sequence[0] != "M":
                        print(translated_sequence)
#                             print(start, stop)
                        if complement == "0":
                            start = int(start) - 3
                        if complement == "1":
                            stop = int(stop) + 3
                        old_trans = translated_sequence
                    if "*" in translated_sequence:
                        error = "Y"
                        break
                    if translated_sequence[0] == "M":
                        break

#Check for stop at the end of last exon

                if query_name == query_name_list[-1]:
                    if translated_sequence[-1] != "*":
                        print(translated_sequence)
#                             print(start, stop)
                        if complement == "0":
                            stop = int(stop) + 3
                        if complement == "1":
                            start = int(start) - 3
                        old_trans = translated_sequence
                    if translated_sequence[-1] == "*":

                        break
            else:
                break
        
        
#Add offset at the beginning and end of each exon
        if (start != 0 or stop != 0):
            
            start_modifier = int(query_name.split("Frame")[1][1])
            stop_modifier = int(query_name.split("rightoh")[1][1])
        else:
            start_modifier = 0
            stop_modifier = 0            

        seq_length = query_length
        
        
#Get exon start and end by adding or removing codons
        
    #if not the beginning
        if query_start_coor != "1" and query_name != query_name_list[0]:
            if complement == "0":
                start = int(start) - 3*(int(query_start_coor)-1)                
            if complement == "1":
                stop = int(stop) + 3*(int(query_start_coor)-1)
    
    #For the end
        if query_stop_coor != str(seq_length) and query_name != query_name_list[-1]:
            if complement == "0":
                stop = int(stop) + 3*(int(seq_length)-int(query_stop_coor))
            if complement == "1":
                

                start = int(start) - 3*(int(seq_length)-int(query_stop_coor))
 
        
        
        
#Adding or removing 3' and 5' overhangs for forward and reverse complement
    #For forward complement
        if complement == "0":
            start = int(start) - int(start_modifier)
            stop = int(stop) +  int(stop_modifier)
            if old_end != 0 and old_end > stop:

                error = "Y"
            old_end = stop

    #For reverse complement
        if complement == "1":
            start = int(start) - int(stop_modifier)
            stop = int(stop) +  int(start_modifier)
            if old_end != 0 and old_end < stop:
                error = "Y"
            old_end = stop

    #Simple check for lenghth
        if start == 0 or stop == 0:
            error = "Y"
            
           
        output_format = str(species.split("\n")[0])+"," + str(scaff) +"," + str(start)+"," + str(stop)+"," + str(complement)+"," + str(error)+  ","+ str(query_name)+","+ str(query_start_coor)+","+str(query_stop_coor)+","+str(query_length)+ "\n"  
        print(output_format)
#         assert False
        Output_Sequence = Output_Sequence + output_format

    print(Output_Sequence)
   
    output_file = open(f"{blast_location}/1.Blast_result/{species}/{species}_coordinates_old.csv",'w')
    output_file.write(Output_Sequence)
    output_file.close()


Danaus_plexippus
Hestina_assimilis,Intial_value,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_2_query_Exon_1,0,0,12

Danaus_plexippus
Hestina_assimilis,CM041759.1,30882230,30882378,0,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_2,1,49,49

Danaus_plexippus
Hestina_assimilis,CM041759.1,30882451,30882632,0,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_0_query_Exon_3,1,60,60

Danaus_plexippus
Hestina_assimilis,CM041759.1,30882712,30882826,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,2,26,38

Danaus_plexippus
Hestina_assimilis,CM041759.1,30885911,30886006,0,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31

Danaus_plexippus
Hestina_assimilis,CM041759.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,0,0,19

Danaus_plexippus
Hestina_assimilis,CM041759.1,30888218,30888456,0,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_7,1,79,79

Danaus_plexippus
Hestina_assimilis,CM041759.1,30

Danaus_plexippus
Danaus_chrysippus,Intial_value,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_2_query_Exon_1,0,0,12

Danaus_plexippus
Danaus_chrysippus,CAKASE010000043.1,3313971,3314119,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_2,1,49,49

Danaus_plexippus
Danaus_chrysippus,CAKASE010000043.1,3313674,3313855,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_0_query_Exon_3,1,60,60

Danaus_plexippus
Danaus_chrysippus,CAKASE010000043.1,3313474,3313591,1,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,1,38,38

Danaus_plexippus
Danaus_chrysippus,CAKASE010000043.1,3312100,3312195,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31

Danaus_plexippus
Danaus_chrysippus,CAKASE010000043.1,3311806,3311866,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,1,19,19

Danaus_plexippus
Danaus_chrysippus,CAKASE010000043.1,3309962,3310200,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_7,1,79,79

Dan

Dircenna_loreta,OY743092.1,4409508,4409636,1,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_0_query_Exon_13,1,42,42

Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Dircenna_loreta,Intial_value,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_2_query_Exon_1,0,0,12
Dircenna_loreta,OY743092.1,4423435,4423583,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_2,1,49,49
Dircenna_loreta,OY743092.1,4422955,4423136,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_0_query_Exon_3,1,60,60
Dircenna_loreta,OY743092.1,4422735,4422852,1,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,1,38,38
Dircenna_loreta,OY743092.1,4420783,4420878,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31
Dircenna_loreta,OY743092.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,0,0,19
Dircenna_loreta,OY743092.1,4416281,4416519,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_7,

Danaus_plexippus
Melinaea_marsaeus_rileyi,Intial_value,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_2_query_Exon_1,0,0,12

Danaus_plexippus
Melinaea_marsaeus_rileyi,OU911095.1,15458857,15459005,0,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_2,1,49,49

Danaus_plexippus
Melinaea_marsaeus_rileyi,OU911095.1,15459264,15459445,0,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_0_query_Exon_3,1,60,60

Danaus_plexippus
Melinaea_marsaeus_rileyi,OU911095.1,15459523,15459640,0,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,1,38,38

Danaus_plexippus
Melinaea_marsaeus_rileyi,OU911095.1,15461757,15461852,0,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31

Danaus_plexippus
Melinaea_marsaeus_rileyi,OU911095.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,0,0,19

Danaus_plexippus
Melinaea_marsaeus_rileyi,OU911095.1,15469413,15469651,0,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_7,1,79,79


Danaus_plexippus
Napeogenes_inachia,Intial_value,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_2_query_Exon_1,0,0,12

Danaus_plexippus
Napeogenes_inachia,OY365800.1,5420118,5420266,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_2,1,49,49

Danaus_plexippus
Napeogenes_inachia,OY365800.1,5419621,5419802,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_0_query_Exon_3,1,60,60

Danaus_plexippus
Napeogenes_inachia,OY365800.1,5419099,5419216,1,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,1,34,38

Danaus_plexippus
Napeogenes_inachia,OY365800.1,5417128,5417223,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31

Danaus_plexippus
Napeogenes_inachia,OY365800.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,0,0,19

Danaus_plexippus
Napeogenes_inachia,OY365800.1,5410992,5411230,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_7,1,79,79

Danaus_plexippus
Napeogenes_inachia,OY365800.1,5410

...

Danaus_plexippus
Melitaea_cinxia,NC_059424.1,8955192,8955373,0,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_0_query_Exon_3,1,60,60

Danaus_plexippus
Melitaea_cinxia,NC_059424.1,8955458,8955575,0,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,1,38,38

Danaus_plexippus
Melitaea_cinxia,NC_059424.1,8963331,8963426,0,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31

Danaus_plexippus
Melitaea_cinxia,NC_059424.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,0,0,19

Danaus_plexippus
Melitaea_cinxia,NC_059424.1,8970704,8970942,0,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_7,1,79,79

Danaus_plexippus
Melitaea_cinxia,NC_059424.1,8971534,8971681,0,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_8,1,48,48

Danaus_plexippus
Melitaea_cinxia,NC_059424.1,8973008,8973191,0,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_0_query_Exon_9,1,62,62

Danaus_plexippus
Melitaea_cinxia,NC_059424.1,8974363,8

Danaus_plexippus
Nymphalis_c-album,OY720586.1,11563952,11564069,0,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,2,38,38

Danaus_plexippus
Nymphalis_c-album,OY720586.1,11566259,11566354,0,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31

Danaus_plexippus
Nymphalis_c-album,OY720586.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,0,0,19

Danaus_plexippus
Nymphalis_c-album,OY720586.1,11571346,11571584,0,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_7,1,79,79

Danaus_plexippus
Nymphalis_c-album,OY720586.1,11571943,11572090,0,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_8,1,48,48

Danaus_plexippus
Nymphalis_c-album,OY720586.1,11572607,11572790,0,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_0_query_Exon_9,1,62,62

Danaus_plexippus
Nymphalis_c-album,OY720586.1,11573487,11573660,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_0_query_Exon_10,3,57,63

Danaus_plexippus
Nymphalis_c-album

Danaus_plexippus
Nymphalis_io,NC_065918.1,6534069,6534250,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_0_query_Exon_3,1,60,60

Danaus_plexippus
Nymphalis_io,NC_065918.1,6533871,6533988,1,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,1,38,38

Danaus_plexippus
Nymphalis_io,NC_065918.1,6531404,6531499,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31

Danaus_plexippus
Nymphalis_io,NC_065918.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,0,0,19

Danaus_plexippus
Nymphalis_io,NC_065918.1,6525765,6526003,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_7,1,79,79

Danaus_plexippus
Nymphalis_io,NC_065918.1,6525444,6525591,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_8,1,48,48

Danaus_plexippus
Nymphalis_io,NC_065918.1,6524620,6524803,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_0_query_Exon_9,1,62,62

Danaus_plexippus
Nymphalis_io,NC_065918.1,6523605,6523784,1,N,Danaus_plexip

Danaus_plexippus
Nymphalis_urticae,LR989982.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_0_query_Exon_13,0,0,42

Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Nymphalis_urticae,Intial_value,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_2_query_Exon_1,0,0,12
Nymphalis_urticae,LR989982.1,6508152,6508300,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_2,1,49,49
Nymphalis_urticae,LR989982.1,6507886,6508067,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_0_query_Exon_3,1,60,60
Nymphalis_urticae,LR989982.1,6507686,6507803,1,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,1,38,38
Nymphalis_urticae,LR989982.1,6505105,6505200,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31
Nymphalis_urticae,LR989982.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,0,0,19
Nymphalis_urticae,LR989982.1,6499702,6499940,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rig

Danaus_plexippus
Vanessa_tameamea,Intial_value,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_2_query_Exon_1,0,0,12

Danaus_plexippus
Vanessa_tameamea,NW_020662887.1,733261,733409,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_2,1,49,49

Danaus_plexippus
Vanessa_tameamea,NW_020662887.1,732979,733160,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_0_query_Exon_3,1,60,60

Danaus_plexippus
Vanessa_tameamea,NW_020662887.1,732784,732901,1,N,Danaus_plexippus_XM_032670980.2_Frame_0_rightoh_1_query_Exon_4,1,38,38

Danaus_plexippus
Vanessa_tameamea,NW_020662887.1,731130,731225,1,N,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_1_query_Exon_5,1,31,31

Danaus_plexippus
Vanessa_tameamea,NW_020662887.1,0,0,0,Y,Danaus_plexippus_XM_032670980.2_Frame_2_rightoh_2_query_Exon_6,0,0,19

Danaus_plexippus
Vanessa_tameamea,NW_020662887.1,727058,727296,1,N,Danaus_plexippus_XM_032670980.2_Frame_1_rightoh_1_query_Exon_7,1,79,79

Danaus_plexippus
Vanessa_tameamea,NW_020662887.1,72